In [1]:
import sys
sys.path.append('../modules/')
import pandas as pd
import matplotlib.pyplot as plt
import pulp
from mymodule_2 import *
from mymodule2_2 import *
from pulp_func_2 import *
df2023_result = pd.read_excel('../excels/附件3/2023_result.xlsx')
# 去除列名末尾的空格。很奇怪
df2023_result.columns = [column.rstrip(' ') for column in df2023_result.columns]
# print(df2023_result)

In [2]:
# 数据准备

merged_df = get_merged_df(df21, df22)
# print(merged_df)
total_yield_2023 = merged_df.groupby(['作物编号', '作物名称'])['单地块类型总产量/斤'].sum().reset_index()
total_yield_2023.columns = ['作物编号', '作物名称', '总产量/斤']
# print(total_yield_2023)
expected_sales_data_2023 = total_yield_2023.copy()
expected_sales_data_2023.columns = ['作物编号', '作物名称', '预期销售量/斤']
expected_sales_data = total_yield_2023.copy() # 初始值为2023年的数据
# print(expected_sales_data)
expected_sales_data.columns = ['作物编号', '作物名称', '预期销售量/斤']

# 按照 '作物编号' 和 '作物名称' 进行分组，并对 '利润' 进行求和
total_profit = merged_df.groupby(['作物编号', '作物名称'])[['利润', '每亩利润']].sum().reset_index()
# 重命名列，以便清晰地表示数据
total_profit.columns = ['作物编号', '作物名称', '总利润/元', '每亩总利润/元']
# 打印 total_profit
# print(total_profit)
total_profit_2023 = total_profit['总利润/元'].sum()
print("总利润/元的和:", total_profit_2023)
std_devs = [0.005, 0.005, 0.01, 0.01, 0.003, 0.003, 0.003, 0.003]
past_profits = [total_profit_2023]

总利润/元的和: 5926348.25


In [3]:
fields = create_fields(df11)
# for field in fields:
#     print(field)
new_fields = create_new_fields(fields)
# for field in new_fields:
#     print(field)
crops = create_crops(df12, df22)
# for crop in crops:
#     print(crop)    

### 2 - 2024

In [4]:
# 更新数据
expected_sales_data = update_expected_sales_data(expected_sales_data, expected_sales_data_2023, std_devs)
# print(expected_sales_data)
crops = update_crops(crops, std_devs)
# for crop in crops:
#     print(crop)

In [5]:
variables, binary_variables, actual_sales, excess_yield = create_variables(new_fields, crops)
# 创建线性规划问题
prob = pulp.LpProblem("Crop_Planting_Optimization", pulp.LpMaximize)
# 定义目标函数的各个部分
k = 0.5
min_area_percent = 0.2
max_plots = 4
profit = define_objective_function(variables, actual_sales, excess_yield, crops, new_fields, k)
# 将目标函数添加到问题中
prob += profit
# print(profit)
prob = add_constraints2(prob, variables, binary_variables, actual_sales, excess_yield,
                        crops, new_fields, total_yield_2023, df2023_result, min_area_percent, max_plots)

# 求解问题
prob.solve()
print("Status:", pulp.LpStatus[prob.status])
update_new_fields(variables, new_fields)

objective_value = pulp.value(prob.objective)
print("Objective Function Value:", objective_value)
past_profits.append(objective_value)
scores = [0, 0, 0, 0, 0, 0, 0]
weights = [0.9, 0.1]
stability = calculate_stability(past_profits)
# print("Stability:", stability)
if scores[0] < calculate_score(objective_value, stability, weights):
    scores[0] = calculate_score(objective_value, stability, weights)
    df_template = pd.read_excel('../excels/附件3/template.xlsx')
    output(df_template, 2024, new_fields, k)
# print(df_template)
# print("Scores:", scores)
# 清空当年的种植信息
for field in new_fields:
    field.planted_crop = []
    # print(field.planted_crop)

Status: Infeasible
Objective Function Value: 8754100.7217511


e:\Users\Timothy\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


### 2 - 2025-2030

In [6]:
for year in range(2025, 2027):
    k = 0.5
    min_area_percent = 0.2
    max_plots = 4
    list = [k, min_area_percent, max_plots, year]

    expected_sales_data = update_expected_sales_data(expected_sales_data, expected_sales_data_2023, std_devs)
    crops = update_crops(crops, std_devs)
    # print(expected_sales_data)
    # print(crops)
    df_last1_result = pd.read_excel(f'../excels/附件3/{year - 1}_result_{k}.xlsx')
    df_last1_result.columns = [column.rstrip(' ') for column in df_last1_result.columns]
    if year - 2 == 2023:
        df_last2_result = pd.read_excel(f'../excels/附件3/2023_result.xlsx')
    else:    
        df_last2_result = pd.read_excel(f'../excels/附件3/{year - 2}_result_{k}.xlsx')
    df_last2_result.columns = [column.rstrip(' ') for column in df_last2_result.columns]
    optimize_planting_strategy(new_fields, crops, expected_sales_data, df_last1_result, df_last2_result, list)
    objective_value = pulp.value(prob.objective)
    print("Objective Function Value:", objective_value)

Status: Infeasible


e:\Users\Timothy\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Objective Function Value: 8754100.7217511
Status: Infeasible


e:\Users\Timothy\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


Objective Function Value: 8754100.7217511
